In [1]:
# Install required libraries
!pip install -q kagglehub streamlit pyngrok pandas matplotlib plotly seaborn tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 907.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [2]:
import kagglehub
import pandas as pd

# Download the dataset
path = kagglehub.dataset_download("ananthr1/weather-prediction")
data_path = path + '/seattle-weather.csv'

# Load the dataset
data = pd.read_csv(data_path)
data.dropna(inplace=True)  # Remove missing values
data['date'] = pd.to_datetime(data['date'])  # Convert date to datetime
print(data.head())

100%|██████████| 11.5k/11.5k [00:00<00:00, 12.9MB/s]

Extracting files...
        date  precipitation  temp_max  temp_min  wind  weather
0 2012-01-01            0.0      12.8       5.0   4.7  drizzle
1 2012-01-02           10.9      10.6       2.8   4.5     rain
2 2012-01-03            0.8      11.7       7.2   2.3     rain
3 2012-01-04           20.3      12.2       5.6   4.7     rain
4 2012-01-05            1.3       8.9       2.8   6.1     rain


In [3]:
import plotly.express as px

# Line plot for temperatures
fig = px.line(data, x='date', y=['temp_max', 'temp_min'],
              labels={'value': 'Temperature (°C)', 'date': 'Date'},
              title='Daily Max and Min Temperatures')
fig.show()

# Weather type distribution
weather_counts = data['weather'].value_counts().reset_index()
weather_counts.columns = ['Weather Type', 'Count']
fig = px.bar(weather_counts, x='Weather Type', y='Count', title='Weather Type Distribution')
fig.show()

In [4]:
import numpy as np

# Extract temperature column and reshape
training = data['temp_max'].values.reshape(-1, 1)

# Sliding window function
def df_to_XY(data, window_size=10):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

WINDOW_SIZE = 10
X, y = df_to_XY(training, WINDOW_SIZE)

# Train-validation-test split
X_train, y_train = X[:800], y[:800]
X_val, y_val = X[800:1000], y[800:1000]
X_test, y_test = X[1000:], y[1000:]

# Reshape for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [5]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Define the model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)

# Save the model
model.save('lstm_weather_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 48ms/step - loss: 248.4206 - val_loss: 258.4142
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 111.1698 - val_loss: 172.4727
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 78.8668 - val_loss: 142.6570
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - loss: 66.4660 - val_loss: 122.6959
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - loss: 63.0377 - val_loss: 107.4405
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 53.8181 - val_loss: 97.3773
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 53.1062 - val_loss: 90.1247
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 55.8090 - val_loss: 84.8614
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 54.6223 - val_loss: 82.3895
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 54.3368 - val_loss: 79.7388
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - loss: 51.5511 - val_loss: 78.1127
Epoch 12/50
25/25 ━━━━━━━━━━━━━━━

In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Predict on validation and test sets
y_pred_val = model.predict(X_val).flatten()
y_pred_test = model.predict(X_test).flatten()

# Calculate MAE and RMSE
mae = mean_absolute_error(y_val, y_pred_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))

print(f"Validation MAE: {mae:.2f} °C")
print(f"Validation RMSE: {rmse:.2f} °C")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Validation MAE: 2.79 °C
Validation RMSE: 3.55 °C


In [12]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import plotly.express as px
from keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the trained model
model = load_model('lstm_weather_model.h5')

# Streamlit Configuration
st.set_page_config(page_title="Seattle Weather Predictor", page_icon="🌤️", layout="wide")

# Custom CSS for UI styling
st.markdown("""
    <style>
    .main {
        background-color: #f7f9fc;
        font-family: 'Arial', sans-serif;
    }
    h1, h2, h3, h4 {
        color: #2c3e50;
    }
    .stButton button {
        background-color: #3498db;
        color: white;
        border-radius: 5px;
        padding: 8px 20px;
        font-size: 16px;
        margin: 10px 0px;
    }
    .stMetric {
        font-weight: bold;
    }
    footer {
        visibility: hidden;
    }
    </style>
""", unsafe_allow_html=True)

# Title Section
st.title("🌤️ FutureTemp Weather Predictor")
st.markdown("""
Welcome to the **FutureTemp**! This tool uses **LSTM Neural Networks** to forecast the temperature based on historical data.
Enjoy a visually appealing and interactive experience. 🚀
""")

# Sidebar Section for Input
st.sidebar.header("🔧 Configure Inputs")
st.sidebar.markdown("Adjust the input parameters below:")
window_size = st.sidebar.slider("Number of Days for Prediction", min_value=5, max_value=20, value=10)
inputs = []
for i in range(window_size):
    inputs.append(st.sidebar.number_input(f"Day {i+1} Temperature (°C):", value=10.0))

# Prediction and Metrics Section
if st.sidebar.button("🌡️ Predict Temperature"):
    input_data = np.array(inputs).reshape(1, -1, 1)
    prediction = model.predict(input_data)[0][0]

    # Simulate actual temperature for metrics (replace with real data if available)
    actual_temp = [input_data[0, -1, 0] + np.random.uniform(-2, 2)]

    # Calculate Metrics
    mae = mean_absolute_error(actual_temp, [prediction])
    rmse = np.sqrt(mean_squared_error(actual_temp, [prediction]))
    accuracy = 100 - (abs(actual_temp[0] - prediction) / abs(actual_temp[0]) * 100)

    # Result Cards
    st.markdown("## 📊 Results")
    st.success(f"🌡️ **Predicted Temperature**: {prediction:.2f} °C")
    st.info(f"📏 **Simulated Actual Temperature**: {actual_temp[0]:.2f} °C")

    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("Mean Absolute Error (MAE)", f"{mae:.2f} °C")
    with col2:
        st.metric("Root Mean Squared Error (RMSE)", f"{rmse:.2f} °C")
    with col3:
        st.metric("Prediction Accuracy (%)", f"{accuracy:.2f} %")

    # Graph Section
    st.markdown("## 📈 Temperature Visualization")
    df_plot = pd.DataFrame({
        'Day': range(len(inputs) + 1),
        'Temperature': inputs + [actual_temp[0]],
        'Type': ['Input'] * len(inputs) + ['Actual']
    })
    df_plot.loc[len(df_plot) - 1, 'Type'] = 'Prediction'

    fig = px.line(df_plot, x='Day', y='Temperature', color='Type',
                  title="Temperature Predictions vs Actual",
                  labels={'Temperature': 'Temperature (°C)', 'Day': 'Day'},
                  template="plotly_white")
    st.plotly_chart(fig, use_container_width=True)

# Footer Section
st.markdown("""
    ---
    Made with ❤️ by ** by Boss 👦🏻 Ice 🧊 Film 🎞️**
    """)

Overwriting app.py


In [13]:
from pyngrok import ngrok
import subprocess

# Set ngrok authentication
ngrok.set_auth_token("2pfjrub3RYXR8RgEVcqkK4dtJVq_4jw4yz6Nt9KkLxnwBJ2Kb")

# Run Streamlit app
process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Expose the app with ngrok
public_url = ngrok.connect(8501)
print(f"Streamlit app running at: {public_url}")

Streamlit app running at: NgrokTunnel: "https://30ee-35-185-23-141.ngrok-free.app" -> "http://localhost:8501"
